## Import and load keys

In [114]:
import os
import requests
from dotenv import load_dotenv
from mistralai import Mistral
import json
import gradio as gr
import random
import time
import traceback

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Obtener la clave de API de Mistral desde las variables de entorno
api_key = os.getenv('MISTRAL_API_KEY')
client = Mistral(api_key=api_key)
model = "mistral-small-latest"

## World Creation

In [115]:
system_prompt = f"""
Your job is to help create interesting fantasy worlds that \
players would love to play in.
Instructions:
- Only generate in plain text without formatting.
- Use simple clear language without being flowery.
- You must stay below 3-5 sentences for each description.
"""

world_prompt = f"""
Generate a creative description for a unique fantasy world with an
interesting concept around cities build on the backs of massive beasts.

Output content in the form:
World Name: <WORLD NAME>
World Description: <WORLD DESCRIPTION>

World Name:"""


output = client.chat.complete(
    model= model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_prompt}
    ],
    temperature=0.9
)

In [116]:
world_output =output.choices[0].message.content
print(world_output)
world_output = world_output.strip()
world = {
    "name": world_output.split('\n')[0].strip()
    .replace('World Name: ', ''),
    "description": '\n'.join(world_output.split('\n')[1:])
    .replace('World Description:', '').strip()
}

World Name: The Skyborne Realms of Aetheria

World Description: In Aetheria, cities float on the backs of colossal, docile beasts called Aetherwhales, which glide through the endless skies. These cities, known as Skyhavens, are bustling metropolises with towering spires and lush gardens, connected by bridges and aerial walkways. The people of Aetheria, called Aetherians, have adapted to this unique environment, using wind magic and advanced engineering to navigate the skies. Below, the surface world is a mysterious and dangerous place, filled with ancient ruins and formidable creatures.


## Kingdom Creation

In [117]:
kingdom_prompt = f"""
Create 3 different kingdoms for a fantasy world.
For each kingdom generate a description based on the world it's in. \
Describe important leaders, cultures, history of the kingdom.\

Output content in the form:
Kingdom 1 Name: <KINGDOM NAME>
Kingdom 1 Description: <KINGDOM DESCRIPTION>
Kingdom 2 Name: <KINGDOM NAME>
Kingdom 2 Description: <KINGDOM DESCRIPTION>
Kingdom 3 Name: <KINGDOM NAME>
Kingdom 3 Description: <KINGDOM DESCRIPTION>

World Name: {world['name']}
World Description: {world['description']}

Kingdom 1"""


output = client.chat.complete(
    model= model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": kingdom_prompt}
    ],
    temperature=0.9
)

In [118]:
kingdoms = {}
kingdoms_output = output.choices[0].message.content

for output in kingdoms_output.split('\n\n'):
  kingdom_name = output.strip().split('\n')[0] \
    .split('Name: ')[1].strip()
  print(f'Created kingdom "{kingdom_name}" in {world["name"]}')
  kingdom_description = output.strip().split('\n')[1] \
    .split('Description: ')[1].strip()
  kingdom = {
      "name": kingdom_name,
      "description": kingdom_description,
      "world": world['name']
  }
  kingdoms[kingdom_name] = kingdom
world['kingdoms'] = kingdoms

print(f'\nKingdom 1 Description: \
{kingdom["description"]}')

Created kingdom "Aerion" in The Skyborne Realms of Aetheria
Created kingdom "Lumina" in The Skyborne Realms of Aetheria
Created kingdom "Ignis" in The Skyborne Realms of Aetheria

Kingdom 1 Description: Ignis is a kingdom built around the mastery of fire magic, with forges that burn brighter than the sun. It is governed by the Council of Flames, a group of powerful fire mages who rule with an iron fist. Ignis's culture is one of discipline and strength, with trials of fire being a rite of passage for all citizens. The kingdom's history is shrouded in mystery, with whispers of a ancient pact with a fire entity, granting Ignis its power, but also a constant threat of destruction.


## Town Creation

In [119]:
def get_town_prompt(world, kingdom):
    return f"""
    Create 3 different towns for a fantasy kingdom and world. \
    Describe the region it's in, important places of the town, \
    and interesting history about it. \
    
    Output content in the form:
    Town 1 Name: <TOWN NAME>
    Town 1 Description: <TOWN DESCRIPTION>
    Town 2 Name: <TOWN NAME>
    Town 2 Description: <TOWN DESCRIPTION>
    Town 3 Name: <TOWN NAME>
    Town 3 Description: <TOWN DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description {kingdom['description']}
    
    Town 1 Name:"""
    
import time

import time  # Asegúrate de importar la biblioteca time

def create_towns(world, kingdom):
    print(f'\nCreating towns for kingdom: {kingdom["name"]}...')
    output = client.chat.complete(
      model=model,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": get_town_prompt(world, kingdom)}
      ],
      temperature=0.9,
  )
    towns_output = output.choices[0].message.content
    
    towns = {}
    for output in towns_output.split('\nCreating towns for kingdom: {kingdom["name"]}...'):
        for num in range(1, 4):
            print(output.split(f"Town {num} Name: ")[1].split("\n")[0])
            print(output.split(f"Town {num} Description: ")[1].split("\n")[0])
            print("\n")
         
            town_name = output.split(f"Town {num} Name: ")[1].split("\n")[0] 
            town_description = output.split(f"Town {num} Description: ")[1].split("\n")[0]
        
            town = {
              "name": town_name,
              "description": town_description,
              "world": world['name'],
              "kingdom": kingdom['name']
            }
            towns[town_name] = town
        kingdom["towns"] = towns

In [120]:
for kingdom in kingdoms.values():
    create_towns(world, kingdom)  

first_kingdom = list(kingdoms.values())[0]
print(first_kingdom)
town = list(first_kingdom['towns'].values())[0]
print(f'\nTown 1 Description: {town["description"]}')


Creating towns for kingdom: Aerion...
Zephyria
Zephyria is a bustling town nestled in the heart of the Whispering Winds region, where the air is always filled with a gentle breeze. The town is famous for its grand Wind Temple, a place of worship and learning dedicated to the wind spirits. Zephyria's history is marked by the legendary Wind Singer, a bard who could control the winds with her music, and whose melodies are said to still echo through the town's streets at night. The town's marketplace is a vibrant hub of trade, where merchants sell everything from enchanted wind chimes to rare sky-whale ivory.


Lumynara
Lumynara is a town built on the back of a massive, luminescent Aetherwhale, drifting through the Starlight Expanse. The town is known for its glowing architecture, which casts a soft light on the surrounding clouds. Lumynara's most important place is the Observatory of the Stars, where astronomers study the celestial bodies and their influence on the winds. The town's hist

## Npcs Creation

In [121]:
def get_npc_prompt(world, kingdom, town): 
    return f"""
    Create 3 different characters based on the world, kingdom \
    and town they're in. Describe the character's appearance and \
    profession, as well as their deeper pains and desires. \
    
    Output content in the form:
    Character 1 Name: <CHARACTER NAME>
    Character 1 Description: <CHARACTER DESCRIPTION>
    Character 2 Name: <CHARACTER NAME>
    Character 2 Description: <CHARACTER DESCRIPTION>
    Character 3 Name: <CHARACTER NAME>
    Character 3 Description: <CHARACTER DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description: {kingdom['description']}
    
    Town Name: {town['name']}
    Town Description: {town['description']}
    
    Character 1 Name:"""
    

In [122]:
def create_npcs(world, kingdom, town):
    print(f'\nCreating characters for the town of: {town["name"]}...')
    output = client.chat.complete(
      model= model,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": get_npc_prompt(world, kingdom, town)}
      ],
        temperature=0.7 
    )

    npcs_output = output.choices[0].message.content
    npcs = {}
    for num in range(1, 4):
        print(npcs_output.split(f"Character {num} Name: ")[1].split("\n")[0])
        print(npcs_output.split(f"Character {num} Description: ")[1].split("\n")[0])
        print("\n")

        character_name = npcs_output.split(f"Character {num} Name: ")[1].split("\n")[0]
        character_description = npcs_output.split(f"Character {num} Description: ")[1].split("\n")[0]

        character = {
        "name": character_name,
        "description": character_description,
        "world": world['name'],
        "kingdom": kingdom['name'],
        "town": town['name']
        }
        npcs[character_name] = character
        town["npcs"] = npcs

    
for kingdom in kingdoms.values():
    for town in kingdom['towns'].values():
        create_npcs(world, kingdom, town)
  # For now we'll only generate npcs for one kingdom and town
    break


Creating characters for the town of: Zephyria...
Elara Stormweaver
Elara is a tall, athletic woman with short, silver hair that shimmers like moonlight. She is a renowned wind mage and navigator, her eyes a striking blue that mirrors the endless sky. Elara's heart bears the weight of her family's legacy, torn apart by the great schism, driving her to explore the surface world despite its dangers. She yearns to uncover the truth behind the ancient ruins and the creatures that dwell below, hoping to find a way to heal the rifts within Aerion.


Orion Whisperwind
Orion is a lanky, young man with a mop of curly, dark hair and eyes that sparkle with curiosity. He is an apprentice at the Wind Temple, studying the ancient melodies of the Wind Singer. Orion's life is shadowed by the loss of his mentor, who vanished while researching the surface world. He dreams of becoming a Wind Singer himself, using his music to bridge the gap between Aerion and the mysterious lands below, seeking answers t

## Save World

In [123]:
def save_world(world, filename):
    # Ensure the directory exists
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'w') as f:
        json.dump(world, f)

def load_world(filename):
    with open(filename, 'r') as f:
        return json.load(f) 

save_world(world, f'../shared_data/{world["name"]}.json')

## Generating Text box

In [130]:
demo = None
def start_game(main_loop, share=False):
    # added code to support restart
    global demo
    # If demo is already running, close it first
    if demo is not None:
        demo.close()

    demo = gr.ChatInterface(
        main_loop,
        chatbot=gr.Chatbot(height=250, placeholder="Type 'start game' to begin"),
        textbox=gr.Textbox(placeholder="What do you do next?", container=False, scale=7),
        title="AI RPG",
        # description="Ask Yes Man any question",
        theme="soft",
        examples=["Look around", "Continue the story"],
        cache_examples=False,
                           )
    demo.launch(share=share, 
                server_name="0.0.0.0"
                )

def test_main_loop(message, history):
    return 'Entered Action: ' + message

#start_game(test_main_loop)
start_game(main_loop)

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_4192\1545379410.py:11: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=250, placeholder="Type 'start game' to begin"),
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7860): [winerror 10048] solo se permite un uso de cada dirección de socket (protocolo/dirección de red/puerto)
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7861): [winerror 10048] solo se permite un uso de cada dirección de socket (protocolo/dirección de red/puerto)
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7862): [winerror 10048] solo se permite un uso de cada dirección de socket (protocolo/direcció

* Running on local URL:  http://0.0.0.0:7863

To create a public link, set `share=True` in `launch()`.


## Generating Start

In [131]:
world_name = world["name"]
world = load_world(f'../shared_data/{world_name}.json')
# Access the world, kingdom, town and character. Here is the first of each selected.
#kingdom = random.choice(list(world['kingdoms'].keys()))
kingdom = list(world['kingdoms'].keys())[0] 
town = random.choice(list(world['kingdoms'][kingdom]['towns'].keys()))
character = random.choice(list(world['kingdoms'][kingdom]['towns'][town]['npcs'].keys()))

In [132]:
system_prompt = """You are an AI Game master. Your job is to create a 
start to an adventure based on the world, kingdom, town and character 
a player is playing as. 
Instructions:
You must only use 2-4 sentences \
Write in second person. For example: "You are Jack" \
Write in present tense. For example "You stand at..." \
First describe the character and their backstory. \
Then describes where they start and what they see around them."""
world_info = f"""
World: {world}
Kingdom: {kingdom}
Town: {town}
Your Character: {character}
"""

model_output = client.chat.complete(
    model= model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info + '\nYour Start:'}
    ],
    temperature=1.0
)
start = model_output.choices[0].message.content
print(start)
world['start'] = start
#save_world(world, f'../shared_data/{world_name}.json')  
save_world(world, f'../shared_data/{world_name}.json')

You are Lyra Skyseeker, a middle-aged woman with warm, brown skin and eyes that hold the wisdom of the winds. You are a skilled engineer and inventor, creating marvelous devices that harness the power of the air. Your past is marked by the internal strife caused by the schism, your family divided by their beliefs. You desire to unite Aerion, using your inventions to explore the surface world and prove that cooperation, not conflict, is the key to progress. You stand in your workshop, surrounded by blueprints, gears, and various wind-powered contraptions. The gentle breeze from the Whispering Winds region rustles the papers on your workbench, as the hum of your latest invention fills the air. The scent of oil and metal permeates the space, a testament to your dedication to your craft.


## Creating the Main Action Loop

In [133]:
def run_action(message, history, game_state):
    
    if(message == 'start game'):
        return game_state['start']

    system_prompt = """You are an AI Game master. Your job is to write what \
happens next in a player's adventure game.\
Instructions: \
You must on only write 1-3 sentences in response. \
Always write in second person present tense. \
Ex. (You look north and see...)"""
    
    world_info = f"""
World: {game_state['world']}
Kingdom: {game_state['kingdom']}
Town: {game_state['town']}
Your Character:  {game_state['character']}"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info}
    ]
    for action in history:
        messages.append({"role": "assistant", "content": action[0]})
        messages.append({"role": "user", "content": action[1]})

    messages.append({"role": "user", "content": message})
    model_output = client.chat.complete(
        model=model,
        messages=messages
    )
    
    result = model_output.choices[0].message.content
    return result

In [134]:
game_state = {
    "world": world['description'],
    "kingdom": world["kingdoms"][kingdom]['description'],
    "town": world["kingdoms"][kingdom]["towns"][town]['description'],
    "character": world['kingdoms'][kingdom]['towns'][town]['npcs'][character]['description'],
    "start": start,
}

def main_loop(message, history):
    return run_action(message, history, game_state)

In [135]:
def get_game_state(game_state=game_state, *args, **kwargs):
    world = load_world(f'../shared_data/{world_name}.json')
    kingdom = game_state["kingdom"]
    town = game_state["town"]
    character = game_state["character"]
    start = game_state['start']
    inventory = {}

    game_state = {
        "world": world,
        "kingdom": kingdom,
        "town": town,
        "character": character,
        "start": start,
        "inventory": game_state.get('inventory', {})
    }
    return game_state

## Launch and Share!

In [ ]:
# start_game(main_loop, True)

In [ ]:
# from helper import get_game_state

# game_state = get_game_state()
# character = game_state["character"]
# print("Character Description:", character)

## Define Inventory Detector

In [136]:
system_prompt = """You are an AI Game Assistant. \
Your job is to detect changes to a player's \
inventory based on the most recent story and game state.
If a player picks up, or gains an item add it to the inventory \
with a positive change_amount.
If a player loses an item remove it from their inventory \
with a negative change_amount.
Given a player name, inventory and story, return a list of json update
of the player's inventory in the following form.
Only take items that it's clear the player (you) lost.
Only give items that it's clear the player gained. 
Don't make any other item updates.
If no items were changed return {"itemUpdates": []}
and nothing else.

Response must be in Valid JSON
Don't add items that were already added in the inventory

Inventory Updates:
{
    "itemUpdates": [
        {"name": <ITEM NAME>, 
        "change_amount": <CHANGE AMOUNT>}...
    ]
}
"""

In [137]:
def detect_inventory_changes(game_state, output):
    inventory = game_state.get('inventory', {})
    
    # Prompt más estricto para Mistral
    system_prompt = """You are an inventory tracker. Respond ONLY with valid JSON in this exact format:
{
    "itemUpdates": [
        {"name": "item_name", "change_amount": 1}
    ]
}"""
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Current Inventory: {inventory}"},
        {"role": "user", "content": f"Action: {output}"},
        {"role": "user", "content": "List item changes in the specified JSON format."}
    ]
    
    # Llamada a Mistral
    response = client.chat.complete(
        model=model,
        temperature=0.0,
        messages=messages
    ).choices[0].message.content
    try:
        cleaned = response.strip()
        if cleaned.startswith('```json'):
            cleaned = cleaned[7:-3].strip()
        elif cleaned.startswith('```'):
            cleaned = cleaned[3:-3].strip()
        
        return json.loads(cleaned).get('itemUpdates', [])
    
    except Exception as e:
        print(f"Error parsing inventory: {str(e)}")
        return []


In [138]:
game_state = get_game_state(game_state)
game_state['inventory'] = {
    "cloth pants": 1,
    "cloth shirt": 1,
    "gold": 5
}

result = detect_inventory_changes(game_state, 
"You buy a sword from the merchant for 5 gold")

print(result)

[{'name': 'gold', 'change_amount': -5}, {'name': 'sword', 'change_amount': 1}]


In [139]:
def update_inventory(inventory, updates):
    try:
        if not updates:
            return ""
            
        changes = []
        for item in updates:
            name = item.get('name', '')
            amount = int(item.get('change_amount', 0))
            
            if name and amount != 0:
                inventory[name] = inventory.get(name, 0) + amount
                if inventory[name] <= 0:
                    del inventory[name]
                changes.append(f"{name} ({'+' if amount > 0 else ''}{amount})")
        
        return "Inventario actualizado: " + ", ".join(changes) if changes else ""
    except Exception as e:
        print(f"⚠️ Error en update_inventory: {str(e)}")
        return ""

## Story with inventory

In [140]:
def run_action(message, history, game_state):
    
    if(message == 'start game'):
        return game_state['start']
        
    system_prompt = """You are an AI Game master. Your job is to write what \
happens next in a player's adventure game.\
Instructions: \
You must on only write 1-3 sentences in response. \
Always write in second person present tense. \
Ex. (You look north and see...) \
Don't let the player use items they don't have in their inventory.
"""

    world_info = f"""
World: {game_state['world']}
Kingdom: {game_state['kingdom']}
Town: {game_state['town']}
Your Character:  {game_state['character']}
Inventory: {json.dumps(game_state['inventory'])}"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info}
    ]

    for user_msg, bot_resp in history:
        messages.append({"role": "assistant", "content": bot_resp})
        messages.append({"role": "user", "content": user_msg})
           
    messages.append({"role": "user", "content": message})
    if not message or not history:
        return "Estado del juego no válido"
    model_output = client.chat.complete(
        model=model,
        messages=messages
    )
    
    result = model_output.choices[0].message.content
    return result

In [141]:
game_state = get_game_state(inventory={
    "cloth pants": 1,
    "cloth shirt": 1,
    "goggles": 1,
    "leather bound journal": 1,
    "gold": 5
})

In [143]:
def main_loop(message, history):
    try:
        # Validación básica del input
        if not message.strip():
            return "Por favor escribe una acción válida"
        
        # 1. Ejecutar acción principal
        output = str(run_action(message, history, game_state))  # Aseguramos string
        
        # 2. Validar seguridad
        #if not is_safe(output):
        #    return "⚠️ Esa acción no está permitida en este momento"
        
        # 3. Procesar inventario (con reintentos)
        max_retries = 3
        inventory_msg = ""
        
        for attempt in range(max_retries):
            try:
                item_updates = detect_inventory_changes(game_state, output)
                if item_updates:
                    inventory_msg = update_inventory(game_state['inventory'], item_updates)
                break
            except Exception as e:
                if attempt == max_retries - 1:
                    print(f"⚠️ Fallo al actualizar inventario: {str(e)}")
                time.sleep(1)  # Espera entre reintentos
        
        # Construir respuesta final
        response = output
        if inventory_msg:
            response += f"\n\n🔹 {inventory_msg}"
            
        return response
        
    except Exception as e:
        # Manejo detallado del error
        error_type = type(e).__name__
        
        if str(e) == "0":
            print("🔴 Error crítico: Posible fallo en la API o respuesta vacía")
            return "El sistema no pudo procesar tu acción. Intenta algo diferente."
        else:
            print(f"🔴 Error crítico ({error_type}): {str(e)}")
            return f"Error: {error_type}. Por favor intenta nuevamente."

start_game(main_loop)

Closing server running on port: 7863


C:\Users\USUARIO\AppData\Local\Temp\ipykernel_4192\1545379410.py:11: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=250, placeholder="Type 'start game' to begin"),
c:\Users\USUARIO\AppData\Local\pypoetry\Cache\virtualenvs\llm-game-xAnBDEWD-py3.13\Lib\site-packages\gradio\chat_interface.py:317: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7860): [winerror 10048] solo se permite un uso de cada dirección de socket (protocolo/dirección de red/puerto)
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7861): [winerror

* Running on local URL:  http://0.0.0.0:7863

To create a public link, set `share=True` in `launch()`.


In [ ]:
def check_api_status():
    try:
        test_response = client.chat.complete(
            model=model,
            messages=[{"role": "user", "content": "Responde 'OK'"}],
            temperature=0.0
        )
        return True if test_response.choices[0].message.content else False
    except:
        return False

if not check_api_status():
    print("🔴 La API de Mistral no está respondiendo")
    # Puedes agregar un reintento o salir

check_api_status()

True